# Auto Labeling
This notebook reads the computed fft results for each part and generates the response dataframe for use with machine learning.

In [2]:
%matplotlib inline
import seaborn as sns
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import os
import sys

In [3]:
wav_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognition/parts/'
fft_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognition/fft/'
pos_response_sample_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognitiont/pos-response-samples/'

if not os.path.exists(pos_response_sample_directory):
    os.makedirs(pos_response_sample_directory)

In [4]:
duration = 0.15 # window length in seconds
sample_rate = 16000
sample_size = 2400 # 150 ms window size
stride = 80 # sliding 5 ms window increments

In [5]:
from os.path import isfile, join

def list_files(base_dir, ext):
    onlyfiles = [f for f in os.listdir(base_dir) if isfile(join(base_dir, f)) and f.split('.')[-1] == ext]
    return np.sort(onlyfiles)

wav_files = list_files(wav_directory,'wav')
# Only the following file parts contain the desired signal
pos_wav_files = ['161225-001_16bit-part-9.wav','161225-002_16bit-part-0.wav','161225-003_16bit-part-0.wav','161225-004_16bit-part-0.wav','161225-005_16bit-part-0.wav','161225-006_16bit-part-0.wav','161225-006_16bit-part-1.wav','161225-006_16bit-part-2.wav']

print(wav_files)

['161225-000_16bit-part-0.wav' '161225-000_16bit-part-1.wav'
 '161225-000_16bit-part-2.wav' '161225-000_16bit-part-3.wav'
 '161225-000_16bit-part-4.wav' '161225-000_16bit-part-5.wav'
 '161225-000_16bit-part-6.wav' '161225-001_16bit-part-0.wav'
 '161225-001_16bit-part-1.wav' '161225-001_16bit-part-10.wav'
 '161225-001_16bit-part-2.wav' '161225-001_16bit-part-3.wav'
 '161225-001_16bit-part-4.wav' '161225-001_16bit-part-5.wav'
 '161225-001_16bit-part-6.wav' '161225-001_16bit-part-7.wav'
 '161225-001_16bit-part-8.wav' '161225-001_16bit-part-9.wav'
 '161225-002_16bit-part-0.wav' '161225-003_16bit-part-0.wav'
 '161225-004_16bit-part-0.wav' '161225-005_16bit-part-0.wav'
 '161225-006_16bit-part-0.wav' '161225-006_16bit-part-1.wav'
 '161225-006_16bit-part-2.wav']


In [6]:
fft_compressed_files = list_files(fft_directory,'gz')
print(fft_compressed_files)

['161225-000_16bit-part-0.wav-fft.csv.gz'
 '161225-000_16bit-part-1.wav-fft.csv.gz'
 '161225-000_16bit-part-2.wav-fft.csv.gz'
 '161225-000_16bit-part-3.wav-fft.csv.gz'
 '161225-000_16bit-part-4.wav-fft.csv.gz'
 '161225-000_16bit-part-5.wav-fft.csv.gz'
 '161225-000_16bit-part-6.wav-fft.csv.gz'
 '161225-001_16bit-part-0.wav-fft.csv.gz'
 '161225-001_16bit-part-1.wav-fft.csv.gz'
 '161225-001_16bit-part-10.wav-fft.csv.gz'
 '161225-001_16bit-part-2.wav-fft.csv.gz'
 '161225-001_16bit-part-3.wav-fft.csv.gz'
 '161225-001_16bit-part-4.wav-fft.csv.gz'
 '161225-001_16bit-part-5.wav-fft.csv.gz'
 '161225-001_16bit-part-6.wav-fft.csv.gz'
 '161225-001_16bit-part-7.wav-fft.csv.gz'
 '161225-001_16bit-part-8.wav-fft.csv.gz'
 '161225-001_16bit-part-9.wav-fft.csv.gz'
 '161225-002_16bit-part-0.wav-fft.csv.gz'
 '161225-003_16bit-part-0.wav-fft.csv.gz'
 '161225-004_16bit-part-0.wav-fft.csv.gz'
 '161225-005_16bit-part-0.wav-fft.csv.gz'
 '161225-006_16bit-part-0.wav-fft.csv.gz'
 '161225-006_16bit-part-1.wav-fft

### Each fft row represents the discrete FFT analysis for a 150 ms window of the input signal. Each observation window starts 5 ms apart.

In [7]:
def label_pos_observations(df, x_all, positive_threshold):

    label_window_start = 1100
    label_window_end = 1300
    
    # Ignore the first section of the frequencies as these are not relevant 
    # and have relatively large values compared with the signal of interest.

    cols = np.arange(label_window_start,label_window_end)
    sub_df = df.iloc[:,cols]

    # ### Observations with power values over positive_threshold dB are assigned a True Positive Response
    df_response = pd.DataFrame(data={'response':sub_df.loc[:,cols].max(axis=1) > positive_threshold})
    df_response['x_axis'] = [x for x in np.arange(0,len(x_all)-sample_size,stride)]

    return df_response

def label_neg_observations(df):
    df_response = pd.DataFrame(data={'response':['False']*df.shape[0]})
    df_response['x_axis'] = [x for x in np.arange(0,df.shape[0]*80,stride)]

    return df_response

def save_wav(filename, df, x_all):
    # # Output Training Data
    positive_label_data = df[df['response'] == True].drop('response',axis=1)
    print ('\nPositive Response: %s,%s'%(positive_label_data.shape))

    # Positive Samples
    w_size = stride
    for idx in positive_label_data.index:
        begin = idx * w_size 
        librosa.output.write_wav(pos_response_sample_directory+filename+'positive_sample_%s.wav'%idx,x_all[begin:begin+sample_size], sample_rate)
        
def plot_wav_response(x_all, df):
    fig,ax = plt.subplots(figsize=(12,6))
    ax.plot(x_all)
    df[['x_axis','response']].plot(x='x_axis',y='response',c='r',ax=ax, alpha=0.3)

    plt.xlim(0,1000000)
    plt.ylim(-1,1)
    plt.show()

In [8]:
%%time

threshold = 18

print('Positive Response File Processing...')
for wav_filename,fft_filename in zip(pos_wav_files,[f+'-fft.csv.gz' for f in pos_wav_files]):
    print('\n{0} / {1}'.format(wav_filename,fft_filename))
    df = pd.read_csv(fft_directory+fft_filename)
    cols = np.arange(0,sample_size).tolist()
    df.columns=cols
    
    x_all, sr = librosa.load(wav_directory+wav_filename, sr=sample_rate)
    df_response = label_pos_observations(df,x_all,threshold)
    df_response.to_csv(fft_directory+'{0}-response.csv'.format(fft_filename))
    
    print(df_response['response'].value_counts())

print('Negative Response File Processing...')
neg_files = pd.DataFrame([f if f not in pos_wav_files else None for f in wav_files]).dropna().iloc[:,0].tolist()

for wav_filename,fft_filename in zip(neg_files,[f+'-fft.csv.gz' for f in neg_files]):
    print('\n{0} / {1}'.format(wav_filename,fft_filename))
    df = pd.read_csv(fft_directory+fft_filename)
    cols = np.arange(0,sample_size).tolist()
    df.columns=cols
    
    df_response = label_neg_observations(df)
    df_response.to_csv(fft_directory+'{0}-response.csv'.format(fft_filename))
    
    print(df_response['response'].value_counts())

Positive Response File Processing...

161225-001_16bit-part-9.wav / 161225-001_16bit-part-9.wav-fft.csv.gz
False    11488
True       482
Name: response, dtype: int64

161225-002_16bit-part-0.wav / 161225-002_16bit-part-0.wav-fft.csv.gz
False    9794
True      459
Name: response, dtype: int64

161225-003_16bit-part-0.wav / 161225-003_16bit-part-0.wav-fft.csv.gz
False    11242
True       439
Name: response, dtype: int64

161225-004_16bit-part-0.wav / 161225-004_16bit-part-0.wav-fft.csv.gz
False    11178
True       429
Name: response, dtype: int64

161225-005_16bit-part-0.wav / 161225-005_16bit-part-0.wav-fft.csv.gz
False    9691
True      446
Name: response, dtype: int64

161225-006_16bit-part-0.wav / 161225-006_16bit-part-0.wav-fft.csv.gz
False    11546
True       424
Name: response, dtype: int64

161225-006_16bit-part-1.wav / 161225-006_16bit-part-1.wav-fft.csv.gz
False    11510
True       460
Name: response, dtype: int64

161225-006_16bit-part-2.wav / 161225-006_16bit-part-2.wav-fft.c